In [1]:
import time
import random
import gc

# Set seed.
random.seed(a=100)

# Create our default list.
short_list = list(random.sample(range(1000000), 10))
long_list = list(random.sample(range(1000000), 10000))

## Quicksort
Quicksort is an old and reliable sorting algorithm. It is similar to merge sort in that it is a "divide and conquer" approach. It relies on partitioning the data around a value called the "pivot." For simplicity, this value is often taken to be the last (i=-1) element in the array. The partitioning function rearranges the values in the array such that all the valuse on the left are less than the pivot and all on the right are greater by iterating over the values and swapping where necessary. It finally swaps the value to the right of the partition with the pivot at the i=-1 position to achieve the required arrangement. This function edits the array in place, and returns the index where the pivot ends up.<br><br>
The quicksort function takes an array and a lower and upper index. It calls the partition function and records the returned index of where the pivot ends up. It then calls itself recursively twice; once on the "left" with the lower and pivot-1 indices, and once on the right with the pivit+1 and upper indices.<br><br>
The only part of the algorithm that can be improved is the selection of the pivot. A good one gives good performance while a bad one can give $O(n^{2})$ complexity. In almost any situation, you have to <I>intentionally try</I> to actually run into the worst-case (unless you're sorting a pre-sorted array), so an average case that's acceptable seems to almost always appear in practice.

In [2]:
def partition(A, low, high, pivot=None):
    """Sorts list around pivot such that all values to the left
    are smaller and all to the right are larger. Inserts pivot in between
    partition before returning index of pivot.
    """
    if pivot == None:   #If no pivot is specified
        pivot = A[high] #set pivot to last item in array
    else:
        try:
            pivot_index = A.index(pivot) #get index for pivot
        except:
            pivot = A[high] #if value not in list, initialize as with no pivot
    if A[high] != pivot:
        A[high], A[pivot_index] = A[pivot_index], A[high]
    pivot_index = high
    i = low-1  #i is marker on left hand side
    
    for j in range(low, high): #iterate over every element in list, left to right
        if A[j] <= pivot: #if element is lower or equal to pivot (belongs on left)...
            
            i += 1     #iterate lower index marker
            A[i], A[j] = A[j], A[i]  #swap with value at i to place left of pivot
            
    A[i+1], A[pivot_index] = A[pivot_index], A[i+1] #insert pivot between partition
    return i + 1 #return position of pivot

def quicksort(A, low, high):
    """quicksort! applies partition function to array and itself recursively to
    both sides of partition.
    """
    if high - low <= 1:   #End condition where subpartition length is 1 or 0.
        return
    sep = partition(A, low, high)  #call partition on array at indices, record pivot pos
    quicksort(A, low, sep-1)   #recurse at left partition
    quicksort(A, sep+1, high)   #recurse at right partition
    return
    

In [3]:
arr = [4, 2, 4, 3, 7, 3, 0, 10, 6, 16, 15, 1, 2, 55, 5]
partition(arr, 0, len(arr)-1, pivot=2)
print(arr)

[0, 1, 2, 2, 7, 3, 4, 10, 6, 16, 15, 5, 4, 55, 3]


## Sanity Check: Does it work?
Run the algorithm on the short list and verify it's working.

In [4]:
lst = short_list.copy()
print(lst)
t1 = time.time()
quicksort(lst, 0, len(lst)-1)
print('sorted in: ', time.time()-t1, ' seconds.')
print(lst)

[152745, 481850, 477025, 997948, 808225, 183236, 739784, 412125, 767514, 366725]
sorted in:  0.0007979869842529297  seconds.
[152745, 183236, 366725, 412125, 477025, 481850, 739784, 767514, 808225, 997948]


Looks like it worked. Now for the big check: how does it do on the long list?

In [5]:
lst = long_list.copy()
t1 = time.time()
quicksort(lst, 0, len(lst)-1)
print('sorted in: ', time.time()-t1, ' seconds.')

sorted in:  0.05302000045776367  seconds.


Hey, not bad. Let's finally see if what they say is true: that sorting a pre-sorted list takes a long time.

In [6]:
t1 = time.time()
quicksort(lst, 0, len(lst)-1)
print('sorted in: ', time.time()-t1, ' seconds.')

RecursionError: maximum recursion depth exceeded in comparison

Maximum recursion depth exceeded? Bah! I must increase the recursion depth, then!

In [7]:
import sys
sys.setrecursionlimit(len(long_list))  #set recursion limit to length of long list.

And we'll see if it breaks my computer!

In [8]:
t1 = time.time()
quicksort(lst, 0, len(lst)-1)
print('sorted in: ', time.time()-t1, ' seconds.')

sorted in:  15.503418684005737  seconds.


Sure enough, it does a really bad job on a pre-sorted list. Funny the way that works.<br><br>
A solution to this issue is to randomly select a pivot, rather than always selecting the rightmost value. Let's see if this solves the issue with sorting a pre-sorted list.

In [9]:
def random_quicksort(A, low, high):
    """quicksort! applies partition function to array and itself recursively to
    both sides of partition. This version uses a random number sampled from the
    input array as the pivot.
    """
    if high - low <= 1:   #End condition where subpartition length is 1 or 0.
        return
    pivot = random.choice(A[low:high])
    sep = partition(A, low, high, pivot=pivot)  #call partition on array at indices, record pivot pos
    random_quicksort(A, low, sep-1)   #recurse at left partition
    random_quicksort(A, sep+1, high)   #recurse at right partition
    return

In [10]:
#First, ensure it's working...
lst = short_list.copy()
print(lst)
t1 = time.time()
random_quicksort(lst, 0, len(lst)-1)
print('sorted in: ', time.time()-t1, ' seconds.')
print(lst)

[152745, 481850, 477025, 997948, 808225, 183236, 739784, 412125, 767514, 366725]
sorted in:  0.0005788803100585938  seconds.
[152745, 183236, 366725, 412125, 477025, 481850, 739784, 767514, 808225, 997948]


In [11]:
#Sort the long list...
lst = long_list.copy()
t1 = time.time()
random_quicksort(lst, 0, len(lst)-1)
print('sorted in: ', time.time()-t1, ' seconds.')

sorted in:  0.6430246829986572  seconds.


In [12]:
#And sort it again (pre-sorted)
lst = long_list.copy()
t1 = time.time()
random_quicksort(lst, 0, len(lst)-1)
print('sorted in: ', time.time()-t1, ' seconds.')

sorted in:  0.7108118534088135  seconds.


That seems to solve the problem! Note that the sort takes more than 10x longer than the other method to sort the long list; this is because of the time it takes to run the random select method, which is run many, many times throughout the process.

In [27]:
def median_of_three(input_list):
    """Impliments insertion sort to sort the three values. Returns median value."""
    sorted_ = input_list
    for i in range(len(input_list)):
        j = i
        while j > 0 and sorted_[j-1] > sorted_[j]:
            sorted_[j-1], sorted_[j] = sorted_[j], sorted_[j-1]
            j -= 1
    return sorted_[1]

def mo3_quicksort(A, low, high):
    """quicksort! applies partition function to array and itself recursively to
    both sides of partition. This version uses the median of three method,
    which takes the median between the first, last, and middle (rounded) index
    as the pivot.
    """
    if high - low <= 1:   #End condition where subpartition length is 1 or 0.
        return
    mdn = median_of_three([A[low], A[high], A[int(((high-low)/2)+low)]])
    sep = partition(A, low, high, pivot=mdn)  #call partition on array at indices, record pivot pos
    random_quicksort(A, low, sep-1)   #recurse at left partition
    random_quicksort(A, sep+1, high)   #recurse at right partition
    return

In [28]:
#First, ensure it's working...
lst = short_list.copy()
print(lst)
t1 = time.time()
mo3_quicksort(lst, 0, len(lst)-1)
print('sorted in: ', time.time()-t1, ' seconds.')
print(lst)

[152745, 481850, 477025, 997948, 808225, 183236, 739784, 412125, 767514, 366725]
sorted in:  0.00031447410583496094  seconds.
[152745, 183236, 366725, 412125, 481850, 477025, 739784, 767514, 808225, 997948]


In [33]:
#Sort unsorted list
lst = long_list.copy()
t1 = time.time()
mo3_quicksort(lst, 0, len(lst)-1)
print('sorted in: ', time.time()-t1, ' seconds.')

sorted in:  0.6378076076507568  seconds.


In [34]:
#and sorted list
t1 = time.time()
mo3_quicksort(lst, 0, len(lst)-1)
print('sorted in: ', time.time()-t1, ' seconds.')

sorted in:  0.6164669990539551  seconds.


It still takes a longer time, but I guess the median of three process adds some additional computation as did the random selection process. A more robust implimentation might be able to measure "sortiness" somehow and select an appropriate implimentation.